# Urban Informatics

# Module 12: Spatial networks

**Download, model, analyze, and visualize street networks (and other spatial data) anywhere in the world from OpenStreetMap.**

What you can do with OSMnx:

  - Download street networks anywhere in the world with a single line of code
  - Download other infrastructure network types, place polygons, building footprints, and points of interest
  - Download by city name, polygon, bounding box, or point/address + network distance
  - Download drivable, walkable, bikeable, or all street networks (or pass in custom query filters)
  - Load street network from a local .osm file
  - Visualize street network as a static image or leaflet web map
  - Simplify and correct the network’s topology to clean and consolidate intersections
  - Save networks to disk as shapefiles, GraphML, or node/edge lists
  - Conduct topological and spatial analyses to automatically calculate dozens of indicators
  - Calculate and plot shortest-path routes as a static image or leaflet web map
  - Fast map-matching of points, routes, or trajectories to nearest graph edges or nodes
  - Plot figure-ground diagrams of street networks and/or building footprints
  - Download node elevations and calculate street grades
  - Visualize travel distance and travel time with isoline and isochrone maps
  - Calculate and visualize street bearings and orientations

More info:

  - [OSMnx documentation](https://osmnx.readthedocs.io)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)

In [ ]:
import networkx as nx
import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
ox.config(use_cache=True, log_console=True)
ox.__version__

In [ ]:
# get a graph for some city
G = ox.graph_from_place('Piedmont, California, USA', network_type='drive')
fig, ax = ox.plot_graph(G)

In [ ]:
# what sized area does our network cover in square meters?
G_proj = ox.project_graph(G)
nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
graph_area_m = nodes_proj.unary_union.convex_hull.area
graph_area_m

In [ ]:
# show some basic stats about the network
ox.basic_stats(G_proj, area=graph_area_m, clean_intersects=True, circuity_dist='euclidean')

stats documentation: https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.stats

In [ ]:
# see more stats (mostly topological stuff) with extended_stats
more_stats = ox.extended_stats(G, ecc=True, bc=True, cc=True) #use arguments to turn other toplogical analyses on/off
for key in sorted(more_stats.keys()):
    print(key)

In [ ]:
# pull up some stat's value
more_stats['radius']

In [ ]:
# save graph to disk as shapefile (for GIS) or graphml file (for gephi etc)
ox.save_graph_shapefile(G, filename='mynetwork_shapefile')
ox.save_graphml(G, filename='mynetwork.graphml')

## Visualize street centrality

In [ ]:
# edge closeness centrality: convert graph to line graph so edges become nodes and vice versa
edge_centrality = nx.closeness_centrality(nx.line_graph(G))

In [ ]:
# list of edge values for the orginal graph
ev = [edge_centrality[edge + (0,)] for edge in G.edges()]

# color scale converted to list of colors for graph edges
norm = colors.Normalize(vmin=min(ev)*0.8, vmax=max(ev))
cmap = cm.ScalarMappable(norm=norm, cmap=cm.inferno)
ec = [cmap.to_rgba(cl) for cl in ev]

# color the edges in the original graph with closeness centralities in the line graph
fig, ax = ox.plot_graph(G, bgcolor='k', axis_off=True, node_size=0,
                        edge_color=ec, edge_linewidth=1.5, edge_alpha=1)

## Routing

In [ ]:
# get the nearest network node to each point
orig_node = ox.get_nearest_node(G, (37.828903, -122.245846))
dest_node = ox.get_nearest_node(G, (37.812303, -122.215006))

In [ ]:
# find the route between these nodes then plot it
route = nx.shortest_path(G, orig_node, dest_node, weight='length')
fig, ax = ox.plot_graph_route(G, route, node_size=0)

In [ ]:
# how long is our route in meters?
net_dist = nx.shortest_path_length(G, orig_node, dest_node, weight='length')
net_dist

In [ ]:
# how far is it between these two nodes as the crow flies, straight-line?
sl_dist = ox.great_circle_vec(G.node[orig_node]['y'], G.node[orig_node]['x'],
                              G.node[dest_node]['y'], G.node[dest_node]['x'])
sl_dist

In [ ]:
# route circuity
net_dist / sl_dist

## Get networks other ways

make queries less ambiguous to help the geocoder out if it's not finding what you're looking for

In [ ]:
# make query an unambiguous dict to help the geocoder find specifically what you're looking for
place = {'city' : 'San Francisco',
         'state' : 'California',
         'country' : 'USA'}
G = ox.graph_from_place(place, network_type='drive')
fig, ax = ox.plot_graph(G, fig_height=10, node_size=0, edge_linewidth=0.5)

In [ ]:
# you can get networks anywhere in the world
G = ox.graph_from_place('2nd Arrondissement, Paris, France', buffer_dist=100, network_type='drive_service')
fig, ax = ox.plot_graph(G, node_size=0, edge_linewidth=0.5)

In [ ]:
# or get network by coordinates, bounding box, or any custom polygon shape
# useful when OSM just doesn't have a polygon for the place you want
lewis_hall = (34.019267, -118.283566)
one_mile = 1609 #in meters
G = ox.graph_from_point(lewis_hall, distance=one_mile, network_type='drive')
fig, ax = ox.plot_graph(G, node_size=0)

examples of getting networks by coordinates, bounding box, or any custom polygon shape: https://github.com/gboeing/osmnx-examples/blob/master/notebooks/01-overview-osmnx.ipynb

## Get other infrastructure types

like building footprints or rail or electric grids or neraby businesses

In [ ]:
# building footprints around the empire state building
point = (40.748482, -73.985402) #coordinates of the empire state building
dist = 812 #meters
gdf = ox.footprints_from_point(point=point, distance=dist)
gdf[['name', 'height', 'geometry']].head()

In [ ]:
# plot the building footprints
gdf_proj = ox.project_gdf(gdf)
bbox_proj = ox.bbox_from_point(point=point, distance=dist, project_utm=True)
fig, ax = ox.plot_footprints(gdf_proj, bbox=bbox_proj, bgcolor='#333333', color='w', figsize=(6,6))

In [ ]:
# restaurants near the empire state buildings
gdf = ox.pois_from_point(point=point, distance=dist, amenities=['restaurant'])
gdf[['name', 'cuisine']].dropna().head()

## In-class workshop

Git clone this repo to your desktop: https://github.com/gboeing/osmnx-examples

Pick 2 or 3 notebooks to work through (skip #00).